In [ ]:
import itertools
import pandas as pd
import numpy as np
from transformers import AutoConfig
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

In [ ]:
models = [
    {"model_name":"meta-llama/Llama-2-7b-hf", "model_size":7}, 
    {"model_name":"meta-llama/Llama-2-13b-hf", "model_size":13},
    {"model_name":"codellama/CodeLlama-34b-hf", "model_size":34},
    {"model_name":"meta-llama/Llama-2-70b-hf", "model_size":70}
]

In [ ]:
# for m in models:
#     cfg = AutoConfig.from_pretrained(m['model_name'])
#     m['config'] = cfg

In [ ]:
seqlen = [{"seqlen":256}]

In [ ]:
max_bs = [{"max_bs":None}]

In [ ]:
bs = [{"bs":None}]

In [ ]:
cpu_offloading = [{"cpu_offloading":False}, {"cpu_offloading":True}]

In [ ]:
distrib_type = [{"distrib_type":"FSDP"}, {"distrib_type":"DDP"}]

In [ ]:
ft_type = [{"ft_type":"LoRA"}, {"ft_type":"QLoRA"}]

In [ ]:
# RTX 3090 is not available in cloud providers A5000 also has 24GB memory
gpus = [{"gpu_model":"A5000", "num_gpus":2, "gpu_mem":24, "nvlink":"False"},
        {"gpu_model":"A100-40", "num_gpus":8, "gpu_mem":40, "nvlink":"True"}]

In [ ]:
wandb_link = [{"wandb_link":None}]

In [ ]:
iters = [models, seqlen, max_bs, bs, cpu_offloading, distrib_type, ft_type, gpus,wandb_link]

In [ ]:
experiments = list(itertools.product(*iters))

In [ ]:
len(experiments)

In [ ]:
def flatten_list_of_dicts(l):
    final_d = {}
    for d in l: 
        for k,v in d.items():
            if k in final_d:
                raise ValueError(f"Key {k} exists.")
            final_d[k] = v
    return final_d

In [ ]:
experiments_flat = [flatten_list_of_dicts(exp) for exp in experiments]

In [ ]:
df = pd.DataFrame(experiments_flat)

In [ ]:
# exclude lora ddp
df = df[~((df['ft_type'] == 'LoRA') & (df['distrib_type'] == 'DDP'))].reset_index(drop=True)

In [ ]:
df.shape

In [ ]:
# !pip install gspread
# !pip install gspread-dataframe

In [ ]:
gc = gspread.oauth()

In [ ]:
url = "https://docs.google.com/spreadsheets/d/1JSQbnkwtqPgc-_wqI3LTCJI6jWCaWafubK0ontWR2_Y"
sheet = gc.open_by_url(url)

In [ ]:
worksheet = sheet.get_worksheet_by_id(0)
set_with_dataframe(worksheet, df)